# Abstract

This notebook is used to import and clean the data file 'tracks.csv', which contains roughly 600,000 Spotify tracks and twenty corresponding features. These features include audio information, such a time signature, tempo, danceability, or speechiness, as well as artists, release date, popularity, and length. 

In addition to cleaning, it also features engineers a 'predicted language' column by using the LangDetect library. It writes a new csv file titled 'master_track_data', which is then used in the notebook 'Data_EDA' to perform a full analysis and modeling process. 

# Import Libraries

In [ ]:
import pandas as pd
from langdetect import detect

# Import Data

In [3]:
tracks = pd.read_csv("data/tracks.csv")

In [4]:
pd.set_option('display.max_columns', None)

# Data Cleaning 

In [11]:
#look all missing values in 'name' column
tracks[tracks['name'].isna()]['id_artists'].value_counts()

All rows missing a track name are from the same artist ID: ['0LyfQWJT6nXafLPZqxe9Of']. Using the Spotify API to lookup this id number, I found that it belongs to 'various artists', which doesn't give us much information. Since it is a small portion of our dataset, we can drop these rows. 

In [ ]:
#drop rows with missing track name
tracks.drop(tracks[tracks['name'].isna()].index, axis=0, inplace=True)

In [12]:
# Convert length in milliseconds to length in minutes and seconds
tracks['length_minutes'] = tracks['duration_ms'] // 60000
tracks['length_seconds'] = (tracks['duration_ms'] % 60000) // 1000

# Format the length as minutes:seconds
tracks['length_formatted'] = tracks.apply(lambda row: f"{row['length_minutes']}:{row['length_seconds']:02d}", axis=1)

# Drop unnecessary columns
tracks.drop(['length_minutes', 'length_seconds'], axis=1, inplace=True)

In [13]:
tracks.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,length_minutes,length_seconds,length_formatted
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3,2,6,2:06
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1,1,38,1:38
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5,3,1,3:01
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3,2,56,2:56
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4,2,43,2:43


In [15]:
#clean up artists and id_artists columns
def clean_artists(artists):
    ids = [x.strip() for x in artists.strip('[]').replace("'", '').split(',')]
    
    return ids
    
tracks['id_artists'] = tracks['id_artists'].apply(clean_artists)   
tracks['artists'] = tracks['artists'].apply(clean_artists)

In [16]:
#remove any 'n/a from artists
def remove_na(artists): 
    nas = ['n/a', 'N/A']
    names = [x for x in artists if x not in nas]
    return names 

#apply to artists column
tracks['artists'] = tracks['artists'].apply(remove_na)

In [17]:
#create release_year and decade columns
tracks['release_year'] = tracks['release_date'].apply(lambda x: x[:4])

def make_decade(year):
    decade = year[:3]
    decade = decade + '0'
    return decade

tracks['decade'] = tracks['release_year'].apply(make_decade)

tracks.drop('release_date', axis=1, inplace=True)

In [18]:
tracks.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,length_formatted,release_year,decade
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,[Uli],[45tIt06XoI0Iio4LBEVpls],0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3,2:06,1922,1920
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,[Fernando Pessoa],[14jtPCOoNZwquk5wd9DxrY],0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1,1:38,1922,1920
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,[Ignacio Corsini],[5LiOoJbxVSAMkBS2fUm3X2],0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5,3:01,1922,1920
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,[Ignacio Corsini],[5LiOoJbxVSAMkBS2fUm3X2],0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3,2:56,1922,1920
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,[Dick Haymes],[3BiJGZsyX9sJchTqcSA7Su],0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4,2:43,1922,1920


In [19]:
#separate artists into different categories: 
tracks['artist_name_1'] = tracks['artists'].apply(lambda x: x[0])

def get_artist_2(artists):
    if len(artists) <2:
        artist = None
    else:
        artist = artists[1]
    return artist

tracks['artist_name_2'] = tracks['artists'].apply(get_artist_2)


In [20]:
def get_artist_3(artists):
    if len(artists) <3:
        artist = None
    else:
        artist = artists[2]
    return artist

tracks['artist_name_3'] = tracks['artists'].apply(get_artist_3)

In [21]:
tracks['artist_name_3'].isna().sum() / len(tracks)
#only 6% of songs have more than two artists, so we may consider dropping this column....

0.9425350451158454

In [22]:
#lowercase track names for uniformity
tracks['name'] = tracks['name'].apply(lambda x: x.lower())

In [38]:
# create a function to detect language, or return 'unknown' if unsuccessful
def get_lang(text):
    try:
        lang = detect(text)
    except: 
        lang = 'unknown'
    return lang

tracks['name_language'] = tracks['name'].apply(get_lang)

In [59]:
tracks['name_language'].value_counts()

en       143267
es        46404
it        35394
pt        26420
fr        21722
de        18452
id        18281
fi        16966
tl        15775
da        14962
no        14928
so        14370
tr        13806
ko        13025
af        12280
sv        11347
et        11031
cy        10705
ca        10626
ro         9965
nl         9681
ja         8612
he         8434
pl         7828
sw         7690
hu         7507
sk         6770
sl         6328
th         6289
cs         5529
hr         5446
lt         4973
ru         4895
sq         3562
zh-cn      3272
lv         3153
zh-tw      2258
bg         1433
mk         1211
uk          624
vi          589
el          492
ar          216
fa           24
ur            5
Name: name_language, dtype: int64

In [46]:
#impute tracks with digits to English
condition = (tracks['name_language'] == 'unknown') & (tracks['name'].str.contains(r'\d'))
tracks.loc[condition, 'name_language'] = 'en'

In [50]:
tracks = tracks[tracks['name_language'] != 'unknown']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 586601 entries, 0 to 586671
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                586601 non-null  object 
 1   name              586601 non-null  object 
 2   popularity        586601 non-null  int64  
 3   duration_ms       586601 non-null  int64  
 4   explicit          586601 non-null  int64  
 5   artists           586601 non-null  object 
 6   id_artists        586601 non-null  object 
 7   danceability      586601 non-null  float64
 8   energy            586601 non-null  float64
 9   key               586601 non-null  int64  
 10  loudness          586601 non-null  float64
 11  mode              586601 non-null  int64  
 12  speechiness       586601 non-null  float64
 13  acousticness      586601 non-null  float64
 14  instrumentalness  586601 non-null  float64
 15  liveness          586601 non-null  float64
 16  valence           58

In [60]:
tracks.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,length_formatted,release_year,decade,artist_name_1,artist_name_2,artist_name_3,name_language
0,35iwgR4jXetI318WEWsa1Q,carve,6,126903,0,[Uli],[45tIt06XoI0Iio4LBEVpls],0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3,2:06,1922,1920,Uli,None,None,ro
1,021ht4sdgPcrDgSk7JTbKY,capítulo 2.16 - banquero anarquista,0,98200,0,[Fernando Pessoa],[14jtPCOoNZwquk5wd9DxrY],0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1,1:38,1922,1920,Fernando Pessoa,None,None,es
2,07A5yehtSnoedViJAZkNnc,vivo para quererte - remasterizado,0,181640,0,[Ignacio Corsini],[5LiOoJbxVSAMkBS2fUm3X2],0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5,3:01,1922,1920,Ignacio Corsini,None,None,pt
3,08FmqUhxtyLTn6pAh6bk45,el prisionero - remasterizado,0,176907,0,[Ignacio Corsini],[5LiOoJbxVSAMkBS2fUm3X2],0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3,2:56,1922,1920,Ignacio Corsini,None,None,es
4,08y9GfoqCWfOGsKdwojr5e,lady of the evening,0,163080,0,[Dick Haymes],[3BiJGZsyX9sJchTqcSA7Su],0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4,2:43,1922,1920,Dick Haymes,None,None,en


In [61]:
#write our final dataframe into a new .csv file
tracks.to_csv('data/master_track_data.csv')